Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris) 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "Graphium"
data_name = "qm9"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='graphium', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f"{gcp_root}/data/raw/qm9.csv"
table = pd.read_csv(PATH)
table.columns

Index(['mol_id', 'smiles', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap',
       'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom',
       'u298_atom', 'h298_atom', 'g298_atom'],
      dtype='object')

### Below we specify the meta information of data columns

In [10]:
# Additional meta-data on the column level
annotations = {
    "mol_id": ColumnAnnotation(description="Molecule identifier"),
    "smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "A": ColumnAnnotation(description="A"),
    "B": ColumnAnnotation(description="B"),
    "C": ColumnAnnotation(description="C"),
    "mu": ColumnAnnotation(description="mu"),
    "alpha": ColumnAnnotation(description="alpha"),
    "homo": ColumnAnnotation(description="homo"),
    "lumo": ColumnAnnotation(description="lumo"),
    "gap": ColumnAnnotation(description="gap"),
    "r2": ColumnAnnotation(description="r2"),
    "zpve": ColumnAnnotation(description="zpve"),
    "u0": ColumnAnnotation(description="u0"),
    "u298": ColumnAnnotation(description="u298"),
    "h298": ColumnAnnotation(description="h298"),
    "g298": ColumnAnnotation(description="g298"),
    "cv": ColumnAnnotation(description="cv"),
    "u0_atom": ColumnAnnotation(description="u0_atom"),
    "u298_atom": ColumnAnnotation(description="u298_atom"),
    "h298_atom": ColumnAnnotation(description="h298_atom"),
    "g298_atom": ColumnAnnotation(description="g298_atom"),
}

### Define `Dataset` object

In [11]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [12]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="QM9 contains quantum chemical properties for a relevant, consistent, and comprehensive chemical space of small organic molecules",
    source="https://pubs.acs.org/doi/pdf/10.1021/ci300415d",
    annotations=annotations,
    tags=["Graph", "Quantum chemistry"],
    owner=owner,
    license="CC-BY-4.0",
    user_attributes={"year": "2012"},
    readme=load_readme(f"org-Graphium/{data_name}/{data_name}_readme.md"),
)

In [13]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

2024-07-16 23:26:15.291 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


'gs://polaris-public/polaris-recipes/org-Graphium/qm9/datasets/qm9-v1/dataset.json'

In [14]:
from polaris.hub.client import PolarisHubClient

client = PolarisHubClient()
client.login()

client.upload_dataset(dataset=dataset, access="private", owner=owner)

2024-07-16 23:26:46.327 | INFO     | polaris.hub.client:login:285 - You are already logged in to the Polaris Hub as  (lu@valencediscovery.com). Set `overwrite=True` to force re-authentication.
2024-07-16 23:27:09.130 | SUCCESS  | polaris.hub.client:upload_dataset:631 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/graphium/qm9-v1


{'id': 'JxyFtl6Ejk7EaS2b6CriF',
 'createdAt': '2024-07-17T03:26:47.118Z',
 'deletedAt': None,
 'name': 'qm9-v1',
 'slug': 'qm9-v1',
 'description': 'QM9 contains quantum chemical properties for a relevant, consistent, and comprehensive chemical space of small organic molecules',
 'tags': ['Graph', 'Quantum chemistry'],
 'userAttributes': {'year': '2012'},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'readme': '## Background\nQM9 is a well-known dataset in the field of 3D GNNs. It consists of 19 graph-level quantum properties associated to an energy-minimized 3D conformation of the molecules [1]. It is considered a simple dataset since all the molecules have at most 9 heavy atoms. We chose QM9 in our ToyMix since it is very similar to the larger proposed quantum datasets, PCQM4M_multitask and PM6_83M, but with smaller molecules\n\n## Assay information\n\n\n## Description of readout:\n\n\n## Data resource\n\n',
 'ownerId': 'zMTB7lQiiukqEmLQF7EjT',
 'creatorId':